### Imports

In [30]:
from importlib import reload
import pandas as pd
import numpy as np
import networkx as nx

from os import listdir
from os.path import isfile, join

# Importieren und neu laden (damit aktuelle Änderungen übernommen werden)
from helper import graph_analyzer, text_preprocesser
reload(text_preprocesser)
reload(graph_analyzer)


from helper.text_preprocesser import prepareText, convert_preprocessed_tokens_to_graph
from helper.graph_analyzer import parallelGetDistanceMeasures, parallel_get_betweenness_list

### Optionen setzen

In [38]:
PARALLEL = False
nx.config.backends.parallel.active = True
nx.config.backends.parallel.n_jobs = 4
print(nx.config)

NetworkXConfig(backend_priority=BackendPriorities(algos=[], generators=[]), backends=Config(parallel=ParallelConfig(active=True, backend='loky', n_jobs=4, verbose=0, temp_folder=None, max_nbytes='1M', mmap_mode='r', prefer=None, require=None, inner_max_num_threads=None, backend_params={})), cache_converted_graphs=True, fallback_to_nx=False, warnings_to_ignore=set())


### Dateipfade laden

In [39]:
# Allgemeiner Pfad zu den Daten
DATA_PATH = "data/input/"

# Liste mit Dateinamen von Büchern
file_name_list = [f for f in listdir(DATA_PATH) if isfile(join(DATA_PATH, f))]

# Liste mit Inhalten von Dateien
file_content_list = [" ".join(open(join(DATA_PATH, f)).readlines()) for f in file_name_list]


### Dataframe mit Dateinamen und Inhalt befüllen

In [40]:
# Tabelle erstellen mit Spalten "title" und "content"
dataframe = pd.DataFrame({'title': file_name_list, 'content': file_content_list})


### Text vorbereiten und Graphen erstellen

In [ ]:
dataframe["prepared_text"] = dataframe["content"].apply(prepareText)

dataframe["graph"] = dataframe["prepared_text"].apply(convert_preprocessed_tokens_to_graph)

                             title  \
0   Peter_Camenzind_de_cleaned.txt   
1  Peter_Camenzind_eng_cleaned.txt   

                                             content  \
0  Im Anfang war der Mythus. Wie der große Gott i...   
1  IN THE BEGINNING was the myth. God, in his sea...   

                                       prepared_text  \
0  [im, Anfang, sein, der, Mythus, wie, der, groß...   
1  [in, the, beginning, was, the, myth, god, in, ...   

                                               graph  
0  (im, Anfang, sein, der, Mythus, wie, groß, Got...  
1  (in, the, beginning, was, myth, god, His, Sear...  


### Metriken berechnen

In [43]:
# Basismetriken: Knotenanzahl und Kantenanzahl
dataframe["node_count"] = dataframe["graph"].apply(lambda g: len(g.nodes))
dataframe["edge_count"] = dataframe["graph"].apply(lambda g: len(g.edges))

# Knotengrade
dataframe["degree_list"] = dataframe["graph"].apply(lambda g: np.array([deg for node, deg in g.degree ]))
# Alternative von ChatGPT: dataframe["degree_list"] = dataframe["graph"].apply(lambda g: np.fromiter(dict(g.degree).values(), dtype=float))
dataframe["average_degree"] = dataframe["degree_list"].apply(lambda degree_list: np.mean(degree_list))
dataframe["median_degree"] = dataframe["degree_list"].apply(lambda degree_list: np.median(degree_list))

# Falls eingeschaltet: Parallele Berechnungen
if PARALLEL:
    distance_measures = dataframe["graph"].apply(parallelGetDistanceMeasures).apply(pd.Series)
    dataframe[["diameter", "average_distance"]] = distance_measures
    dataframe["betweenness_list"] = dataframe["graph"].apply(parallel_get_betweenness_list)

    
# Ansonsten (NICHT parallel)
else:
    dataframe["diameter"] = dataframe["graph"].apply(lambda g: nx.diameter(g))
    dataframe["average_distance"] = dataframe["graph"].apply(lambda g: nx.average_shortest_path_length(g))
    dataframe["betweenness_list"] = dataframe["graph"].apply(lambda g : np.array( list(nx.betweenness_centrality(g).values()) ))

# Betweenness aus Liste der Einzelwerte
dataframe["betweenness_min"] = dataframe["betweenness_list"].apply(np.min)
dataframe["betweenness_max"] = dataframe["betweenness_list"].apply(np.max)
dataframe["betweenness_average"] = dataframe["betweenness_list"].apply(np.mean)
dataframe["betweenness_standard_deviation"] = dataframe["betweenness_list"].apply(np.std)


# Clustering
dataframe["average_clustering"] = dataframe["graph"].apply(lambda g: nx.average_clustering(g))


### Speichern der Tabelle in Datei

In [47]:
attributes_visible_in_file = [
    "title",
    "node_count",
    "edge_count",
    "average_degree",
    "median_degree",
    "diameter",
    "average_distance",
    "betweenness_min",
    "betweenness_max",
    "betweenness_average",
    "betweenness_standard_deviation",
    "average_clustering"
]

dataframe[attributes_visible_in_file].to_csv("../olol/output/output.csv", index=False)